# First step is to install the very good ImageDataAugmentor Library from mjkvaak under MIT licence.

In [ ]:
pip install git+https://github.com/mjkvaak/ImageDataAugmentor

# Attempting to predict Sorghum species with pretrained model :

* We will use the given dataset of 22194 1024x1024 images

* We will use data augmentation via Flow_from_dataframe resizing the image 260x260

* We will try EfficientNet B1 in this notebook

# Importing my favorite libraries for Deep learning activities

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import tensorflow as tf
import random
from IPython.core.debugger import set_trace
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.data import Dataset
import cv2
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A
from albumentations.core.composition import Compose, OneOf
import datetime
from ImageDataAugmentor.image_data_augmentor import *

## We define the PATH and Batch_size :

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
PATH = '/kaggle/input/sorghum-id-fgvc-9/'
train_path = PATH+'train_images/'
test_path = PATH+'test/'
batch_size = 16
epoch = 50
WIDTH = 260
HEIGHT = 260

# Create our Dataframe for training :

In [ ]:
image_df = pd.read_csv(PATH+'train_cultivar_mapping.csv')

In [ ]:
image_df.dropna(inplace=True)

In [ ]:
image_df

In [ ]:
# Debugging the notebook with lower values 

# epoch = 20
# image_df = image_df[:200]
# image_df

In [ ]:
kfold = StratifiedKFold(n_splits=4, shuffle=True)

for train_index, valid_index in kfold.split(image_df['image'],image_df['cultivar']):
    train_images, valid_images = image_df['image'].iloc[train_index], image_df['image'].iloc[valid_index]
    train_cultivar, valid_cultivar = image_df['cultivar'].iloc[train_index], image_df['cultivar'].iloc[valid_index]

In [ ]:
train_df= pd.DataFrame({'image':train_images, 'cultivar':train_cultivar})
val_df= pd.DataFrame({'image':valid_images, 'cultivar':valid_cultivar})

In [ ]:
len(train_df), len(val_df)

In [ ]:
# train_df, val_df = train_test_split(image_df, test_size=0.2)

## Images are 1024x1024 RGB in a .png format

# Let's create our Train and Validation datasets (pair of tensors with images preprocessed and targets) :

In [ ]:
transform = Compose([
            A.RandomResizedCrop(height=HEIGHT, width=WIDTH),
            A.Flip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.5),
            A.OneOf([
                A.RandomBrightnessContrast(p=0.5),
                A.RandomGamma(p=0.5),
            ], p=0.5),
            A.OneOf([
                A.Blur(p=0.1),
                A.GaussianBlur(p=0.1),
                A.MotionBlur(p=0.1),
            ], p=0.1),
            A.OneOf([
                A.GaussNoise(p=0.1),
                A.ISONoise(p=0.1),
                A.GridDropout(ratio=0.5, p=0.2),
                A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
            ], p=0.2)
        ])

In [ ]:
train_datagen = ImageDataAugmentor(augment=transform)
val_datagen = ImageDataAugmentor()


In [ ]:
train_augmented = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    shuffle=True,
    directory=train_path,
    x_col='image',
    y_col='cultivar',
    class_mode='categorical',
    target_size=(HEIGHT,WIDTH),
    batch_size=batch_size)

In [ ]:
val_augmented = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    shuffle=True,
    directory=train_path,
    x_col='image',
    y_col='cultivar',
    class_mode='categorical',
    target_size=(HEIGHT,WIDTH),
    batch_size=batch_size)

In [ ]:
num_classes = len(train_augmented.class_indices)
num_classes

In [ ]:
class_id, num_images = np.unique(train_augmented.classes,return_counts=True)
max_value = max(num_images)
class_weights = {c : max_value/n for c,n in zip(class_id, num_images)}

## Let's plot 9 images to see if is works :

In [ ]:
plt.figure(figsize=(10, 10))
batch=train_augmented.next()
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(batch[0][i].astype(np.uint8))
    plt.title(batch[0][i].shape)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 10))
batch=val_augmented.next()
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(batch[0][i].astype(np.uint8))
    plt.title(batch[0][i].shape)
    plt.axis("off")

# It's time to prepare our model and start the training :

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""): # This is the simple function to plot our training and validation curves
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    ax1.set_ylim(0., 4)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val_accuracy'  + exp_name)
    ax2.set_ylim(0, 1)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
def load_model(): # Here we choose our model : Efficientnet B1 pretrained with ImageNet dataset with an input shape of 260x260
    model = tf.keras.applications.EfficientNetB1(include_top=False,weights='imagenet',input_shape=(HEIGHT,WIDTH,3))
    return model

In [ ]:
def set_nontrainable_layers(model): # We define trainability for the base model
    model.trainable=False
    return model

def set_trainable_layers(model): 
    model.trainable=True
    return model

In [ ]:
def add_last_layers(model): # Here we complete our model with last layers for our problem at hand
#     base_model = set_nontrainable_layers(model)
    input_layer = tf.keras.Input(shape=(HEIGHT,WIDTH,3))
    base_model = set_trainable_layers(model)
    flatten_layer = layers.Flatten()
    global_layer = layers.GlobalAveragePooling2D()
    dense_layer = layers.Dense(256, activation='relu', kernel_initializer='he_uniform')
    dropout_layer = layers.Dropout(0.5)
    prediction_layer = layers.Dense(num_classes, activation='softmax')
    
    model = Sequential([
#         input_layer,
        base_model,
        global_layer,
        dropout_layer,
#         flatten_layer,
#         dense_layer,
        prediction_layer
    ])
    return model

In [ ]:
def build_model(): # We assemble our model and compile it with the proper loss function and metrics for image classification
    model = load_model()
    model = add_last_layers(model)
    
    opt = tf.keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                 optimizer=opt,
                 metrics=['accuracy'])
    return model

In [ ]:
model_effnet= build_model()
model_effnet.summary()

In [ ]:
es = EarlyStopping(monitor='val_accuracy',
                   patience=7,
                   verbose=1,
                   restore_best_weights=True)

cp = ModelCheckpoint('/kaggle/working/effnetB1.ckpt',
                     monitor='val_accuracy',
                     verbose=1,
                     save_best_only=True,
                     save_weights_only=False,
                     mode='min' )

csv = tf.keras.callbacks.CSVLogger('history_effnetB1.csv')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.4,
                                                 verbose=1,
                                                 patience=2, 
                                                 min_lr=0.00001)

In [ ]:
STEP_SIZE_TRAIN = train_augmented.n//train_augmented.batch_size
STEP_SIZE_VALID = val_augmented.n//val_augmented.batch_size

In [ ]:
%%time

history = model_effnet.fit(train_augmented,
                    epochs=epoch,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    callbacks=[es,cp,reduce_lr,csv],
                    verbose=1,
                    class_weight=class_weights,
                    validation_data=val_augmented,
                    validation_steps=STEP_SIZE_VALID)

In [ ]:
plot_history(history)

# Now we can use our trained model to predict the labels for test images :

In [ ]:
submission = pd.read_csv(PATH+'sample_submission.csv')
submission

In [ ]:
test_gen= ImageDataAugmentor()

In [ ]:
test_generator = test_gen.flow_from_dataframe(dataframe=submission,
                                              directory=test_path,
                                              x_col='filename',
                                              y_col=None,
                                              target_size=(WIDTH,HEIGHT),
                                              color_mode='rgb',
                                              class_mode=None,
                                              batch_size=1,
                                              shuffle=False,)

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [ ]:
STEP_SIZE_TEST,test_generator.n,test_generator.batch_size

In [ ]:
%time
test_generator.reset()
results = model_effnet.predict(test_generator,verbose=1,steps=STEP_SIZE_TEST)

In [ ]:
# best_results = np.argmax(results,axis=1)
# label_dict = pd.read_csv(PATH+'train_cultivar_mapping.csv').drop(columns=['image'])
# best_cultivar = []
# for result in best_results:
#     best_cultivar.append(label_dict.iloc[result].values[0])
# submission['cultivar']=best_cultivar


In [ ]:
predicted_class_indices=np.argmax(results,axis=1)

In [ ]:
labels = (train_augmented.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames


In [ ]:
submission=pd.DataFrame({"Filename":[filename.replace('all_classes/','')for filename in filenames],
                      "cultivar":predictions})

In [ ]:
submission

# And finally we create our submission.csv in order to scor our prediction :

In [ ]:
submission.to_csv('submission_effnet_dataaug.csv',index=False)

## For the moment, the test accuracy remains very low, I am grateful for any comment or suggestion on how to achieve better accuracy with my code.

# Have a nice day

In [ ]:
!kaggle competitions submit -c sorghum-id-fgvc-9 -f submission_effnet_dataaug.csv -m "With flow from dataframe generator"